In [ ]:
!git clone https://github.com/Xilinx/Vitis-AI.git
%cd Vitis-AI/src/vai_quantizer/vai_q_pytorch/pytorch_binding/
%cd /content/Vitis-AI/src/vai_quantizer/vai_q_onnx


!sh build.sh
!pip install pkgs/*.whl
!ls


In [ ]:
!python setup.py install

In [ ]:

!pip install xir # Replace with the appropriate version if needed
!pip install dill
!pip install Ninja

In [ ]:
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['TORCH_CUDA_ARCH_LIST'] = '7.5'


In [ ]:
%cd ..

In [ ]:
!pip install onnxruntime==1.14.0

In [ ]:
import vai_q_onnx
from vai_q_onnx import CalibrationMethod, VitisQuantFormat, QuantType
import numpy as np

# Custom Calibration Data Reader
class CalibrationDataReader:
    def __init__(self, calibration_data):
        self.calibration_data = calibration_data
        self.data_index = 0

    def get_next(self):
        if self.data_index < len(self.calibration_data):
            input_data = self.calibration_data[self.data_index]
            self.data_index += 1
            return {"images": input_data}  # Using 'images' as input node name
        else:
            return None

# Perform Static Quantization
def quantize_onnx_model_static(model_input_path, model_output_path, calibration_data):
    # Create the calibration data reader
    calibration_data_reader = CalibrationDataReader(calibration_data)

    # Call the quantize_static API for Vitis AI ONNX quantization with QDQ format
    vai_q_onnx.quantize_static(
        model_input=model_input_path,                          # Path to the ONNX model
        model_output=model_output_path,                        # Output path for quantized model
        calibration_data_reader=calibration_data_reader,       # Reader that provides calibration data
        quant_format=VitisQuantFormat.QDQ,                     # QDQ quantization format (recommended for x64 performance)
        calibrate_method=CalibrationMethod.MinMax,             # Calibration method (MinMax)
        input_nodes=["images"],                                # Input node (consistent with ONNX export)
        output_nodes=["output"],                               # Output node
        op_types_to_quantize=None,                             # Optional: Ops to quantize (None = all ops)
        per_channel=False,                                     # Whether to apply per-channel quantization
        reduce_range=False,                                    # Optional: Reduces the quantization range
        activation_type=QuantType.QInt8,                       # Activation quantization type (int8)
        weight_type=QuantType.QInt8,                           # Weight quantization type (int8)
        optimize_model=True,                                   # Whether to optimize the model post-quantization
        use_external_data_format=False,                        # External data format (optional)
        extra_options=None                                     # Extra options (if any)
    )

    print(f"Quantized model saved at: {model_output_path}")

if __name__ == "__main__":
    # Define paths for the input and output ONNX models
    model_input_path = "/content/yolov9.onnx"  # Replace with your ONNX model path
    model_output_path = "/content/quantized_model.onnx"  # Path to save the quantized model

    # Example calibration data (dummy data for illustration purposes)
    # Pre-processing: Normalizing the data (you can add more pre-processing if needed)
    calibration_data = np.random.randn(1, 3, 640, 640).astype(np.float32)
    calibration_data = [calibration_data / 255.]

    # Perform static quantization
    quantize_onnx_model_static(model_input_path, model_output_path, calibration_data)

In [ ]:
!pip install git+https://github.com/Xilinx/pyxir.git

In [ ]:
!pip install netron
import netron
netron.start("/content/quantized_model_last.onnx")

In [ ]:
model = onnx.load("/content/yolov9.onnx")

In [ ]:
for node in model.graph.node:
    print(f"Node name: {node.name}, op_type: {node.op_type}")

In [ ]:
import torch
import torch.quantization
from pytorch_nndct.apis import torch_quantizer, dump_xmodel
import pytorch_nndct as py_nndct

import logging

logging.basicConfig(level=logging.INFO)
%cd yoloModel/


def quantize_torch_model(model_path, output_dir):


    # Assuming your model and quantizer are already initialized...

    quantized_model.train()

    # Set calibration_steps to a larger number if possible.
    calibration_steps = 1000

    # Ensure more unique calibration data
    for i in range(calibration_steps):
        calibration_data = torch.randn([1, 3, 640, 640], dtype=torch.float32)
        quantized_model(calibration_data)

    quantized_model.eval()
    quantized_model.to("cuda")

    with torch.no_grad():
        new_data = new_data.to("cuda")
        output = quantized_model(new_data)
        print(output.shape)